In [1]:
import requests, json, time
from pprint import pprint
import pandas as pd

# Get the API key and username from yaml

import yaml
with open("key.yaml", "r") as file:
    api_keys = yaml.safe_load(file)

api_key = api_keys["API_KEY"]
api_username = "ShubhamG"
defaultHost = "discourse.onlinedegree.iitm.ac.in"

# Headers for authentication
headers = {
    'Api-Key': api_key,
    'Api-Username': api_username
}

In [6]:
# Get the first dataframe of user information
df_user_info = pd.read_csv('./USER_ACTIONS/user_actions.csv')
df_user_info.sample(5, random_state=0)

,subject,user,likes_given,likes_recieved,created_a_new_topic,replied,user's_post_was_quoted,user_marked_a_post_as_solution,Solutions_posted
7,mlt,PulkitMangal,86,578,75,938,0,174,187
2,dlp,21f3002073,33,213,16,179,2,23,23
1,dlp,pkrishna,0,0,0,1,0,0,0
4,maths2,Sannidhi_Alape,0,0,0,1,0,1,1
8,mlt,Vishal,335,520,21,567,0,84,84


In [7]:
# Second dataframe of subject information
df_subject = pd.read_csv('./Subject_specifics/total_posts_per_subject.csv')
df_subject

,subject,total_topics
0,mlt,191
1,maths2,347
2,dlp,107


In [15]:
# Now left-outer join the two tables on the 'subject' column
merged_df = pd.merge(df_user_info, df_subject, on='subject', how='left')

# Rename the "total_topics" column to "overall_topics_count_of_this_subject"
merged_df.rename(columns={'total_topics': 'overall_topics_count_of_this_subject'}, inplace=True)

In [14]:
# Assign the weights to the relevant columns. This can be changed as per the requirement.
weights_dict = { 'likes_given': 0.1, # 0.3
                "likes_recieved": 0.5,
                "created_a_new_topic": 1,
                "replied": 0.8,
                "user's_post_was_quoted": 0.6,
                "user_marked_a_post_as_solution":0.2,
                "Solutions_posted": 1
}
columns_to_be_ignored = ["initial_score",'subject','user','overall_topics_count_of_this_subject', 'normalised_score', 'z_score']

merged_df["initial_score"] = sum(merged_df[column]*weights_dict[column] for column in merged_df.columns if column not in columns_to_be_ignored) # Initial score = sum(column_value*weight)

merged_df["normalised_score"] = merged_df["initial_score"] / merged_df ["overall_topics_count_of_this_subject"] # normalised_score

merged_df["z_score"] = round((merged_df["normalised_score"] - merged_df["normalised_score"].mean()) / merged_df["normalised_score"].std(),2) # z_score

merged_df[["user", "initial_score", "normalised_score", "z_score"]]

,user,initial_score,normalised_score,z_score
0,man,460.9,4.307477,0.24
1,pkrishna,0.8,0.007477,-1.11
2,21f3002073,297.8,2.783178,-0.24
3,lavanya,11.0,0.031700,-1.10
4,Sannidhi_Alape,2.0,0.005764,-1.11
5,23f2000573,813.2,4.257592,0.23
6,Nikita,1697.7,8.888482,1.68
7,PulkitMangal,1344.8,7.040838,1.10
8,Vishal,868.9,4.549215,0.32
